In [ ]:
# ============================================================
# Unified CTRL + PTPN2 CRISPR RL (Model-based + Model-free)
# Omics Codeathon, Boabang Francis
# Encoder:
#   - Align genes (CTRL, PTPN2)
#   - True 80/20 split on PTPN2 BEFORE preprocessing
#   - HVG on PTPN2-TRAIN only → applied to PTPN2-TEST + CTRL
#   - Gene Autoencoder (PTPN2-TRAIN only)
#   - PCA (PTPN2-TRAIN only, applied to all)
#   - Simple inference-only GAT on joint PCA space
#
# RL:
#   - PTPN2: model-based RL with world model f_θ, perturb once at t=0
#   - CTRL: model-free RL, multi-step perturbation (additive updates)
#
# Evaluation:
#   - Single-dim metrics ( MSE, RMSE, MAE, R2, Pearson, AUPRC)
#   - Distance to REAL PTPN2 TRAIN mean
#   - Pseudotime alignment (mapped to PTPN2 pseudotime)
#   - IQCELL-style:
#       * Full H-step trajectories (mean simulated vs experimental)
#       * DTW per latent dimension
#       * 1D Wasserstein (OT) per dimension
#       * DTW heatmap (CTRL vs PTPN2)
#       * Combined similarity table (CSV)
# ============================================================

import numpy as np
import pandas as pd
import scipy.io as sio
from scipy import sparse
from scipy.spatial.distance import cdist
from scipy.stats import wasserstein_distance
import math
import matplotlib.pyplot as plt

import scanpy as sc
import anndata as ad

import gymnasium as gym
from gymnasium import spaces

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_error, r2_score,
    average_precision_score
)
from sklearn.decomposition import PCA

import torch
import torch.nn as nn

from stable_baselines3 import PPO
from sb3_contrib import TRPO
from stable_baselines3.common.env_checker import check_env


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# ============================================================
# 1. PATHS / HYPERPARAMETERS
# ============================================================

CTRL_MATRIX = "GSM3371684_Control_matrix.mtx"
CTRL_FEAT   = "GSM3371684_Control_genes.tsv"

PTPN2_MATRIX = "GSM3371687_aPD1-aCTLA4_matrix.mtx"
PTPN2_FEAT   = "GSM3371687_aPD1-aCTLA4_genes.tsv"

#PTPN2_MATRIX = "GSM3371685_aPD1_matrix.mtx"
#PTPN2_FEAT   = "GSM3371685_aPD1_genes.tsv"


# Encoder
N_TOP_GENES   = 50
GENE_EMB_DIM  = 16
N_PCS         = 10
GAT_OUT_DIM   = 32

# World model
WORLD_HIDDEN  = 128
K_NEXT        = 5
WORLD_EPOCHS  = 100
WORLD_LR      = 1e-3

# RL
#ACTION_SCALE  = 0.3 # For higher Perturbation
HORIZON       =  1  # multi-step length for both systems  (Fixed at one for single Perturbation Experiment)

PPO_TIMESTEPS_BASELINE = 150_000
TRPO_TIMESTEPS_STAGE1  = 10_000
PPO_TIMESTEPS_STAGE2   = 140_000

PPO_TIMESTEPS_BASELINE_ctrl = 150_000
TRPO_TIMESTEPS_STAGE1_ctrl  = 10_000
PPO_TIMESTEPS_STAGE2_ctrl   = 140_000



ROLLOUT_SAMPLES = 1500       # for metrics (start vs final)
N_TRAJ_DTW      = 128        # for DTW/OT trajectory analysis

# Training hyperparams
AE_EPOCHS       = 100
BATCH_SIZE      = 256

ALPHA           = 1 # for CTRL in-silico target


# ============================================================
# 2. LOAD CTRL & PTPN2 MATRICES + ALIGN GENES
# ============================================================

def load_mtx(mtx_path, feat_path):
    X = sio.mmread(mtx_path)
    if not sparse.isspmatrix(X):
        X = sparse.coo_matrix(X)
    X = X.T.tocsr()  # cells x genes
    genes = pd.read_csv(feat_path, sep="\t", header=None).iloc[:, 0].astype(str).values
    return X, genes

ctrl_X_raw, ctrl_genes   = load_mtx(CTRL_MATRIX, CTRL_FEAT)
ptpn2_X_raw, ptpn2_genes = load_mtx(PTPN2_MATRIX, PTPN2_FEAT)

common_genes, idx_ctrl, idx_ptpn2 = np.intersect1d(
    ctrl_genes, ptpn2_genes, return_indices=True
)

ctrl_X_raw  = ctrl_X_raw[:, idx_ctrl]
ptpn2_X_raw = ptpn2_X_raw[:, idx_ptpn2]
genes_common = common_genes

print("CTRL raw:", ctrl_X_raw.shape, "PTPN2 raw:", ptpn2_X_raw.shape)
print("Common genes:", len(genes_common))

X_ctrl   = ctrl_X_raw.toarray().astype(np.float32)
X_ptpn2  = ptpn2_X_raw.toarray().astype(np.float32)


# ============================================================
# 3. TRUE 80/20 PTPN2 SPLIT (BEFORE ANY PREPROCESSING)
# ============================================================

X_ptpn2_train_raw, X_ptpn2_test_raw = train_test_split(
    X_ptpn2, test_size=0.20, random_state=42, shuffle=True
)

print("PTPN2 train raw:", X_ptpn2_train_raw.shape)
print("PTPN2 test raw:",  X_ptpn2_test_raw.shape)


# ============================================================
# 4. HVG ON PTPN2 TRAIN ONLY, APPLY TO TEST + CTRL
# ============================================================

adata_ptpn2_train = ad.AnnData(X_ptpn2_train_raw)
adata_ptpn2_train.var_names = genes_common

# Train-only normalization, HVG, scaling
sc.pp.normalize_total(adata_ptpn2_train, target_sum=1e4)
sc.pp.log1p(adata_ptpn2_train)
sc.pp.highly_variable_genes(adata_ptpn2_train, n_top_genes=N_TOP_GENES, subset=True)
sc.pp.scale(adata_ptpn2_train, max_value=10)

hvg_genes = adata_ptpn2_train.var_names
print("HVG genes (PTPN2 train):", len(hvg_genes))

hvg_mask = np.isin(genes_common, hvg_genes)

# Apply to PTPN2 test
adata_ptpn2_test = ad.AnnData(X_ptpn2_test_raw[:, hvg_mask])
adata_ptpn2_test.var_names = hvg_genes
sc.pp.normalize_total(adata_ptpn2_test, target_sum=1e4)
sc.pp.log1p(adata_ptpn2_test)
sc.pp.scale(adata_ptpn2_test, max_value=10)

# Apply to CTRL
adata_ctrl = ad.AnnData(X_ctrl[:, hvg_mask])
adata_ctrl.var_names = hvg_genes
sc.pp.normalize_total(adata_ctrl, target_sum=1e4)
sc.pp.log1p(adata_ctrl)
sc.pp.scale(adata_ctrl, max_value=10)

print("PTPN2 train HVG:", adata_ptpn2_train.shape)
print("PTPN2 test HVG:",  adata_ptpn2_test.shape)
print("CTRL HVG:",         adata_ctrl.shape)

X_ptpn2_train = adata_ptpn2_train.X.astype(np.float32)
X_ptpn2_test  = adata_ptpn2_test.X.astype(np.float32)
X_ctrl_hvg    = adata_ctrl.X.astype(np.float32)


# ============================================================
# 5. GENE AUTOENCODER (PTPN2 TRAIN ONLY)
# ============================================================

n_ptpn2_train, n_genes = X_ptpn2_train.shape
print("Autoencoder input:", X_ptpn2_train.shape)

class GeneAutoencoder(nn.Module):
    def __init__(self, in_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, in_dim)
        )

    def encode(self, x):
        return self.encoder(x)

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat, z

ae = GeneAutoencoder(n_genes, GENE_EMB_DIM).to(DEVICE)
ae_opt = torch.optim.Adam(ae.parameters(), lr=1e-3)
ae_loss_fn = nn.MSELoss()

X_ptpn2_train_tensor = torch.tensor(X_ptpn2_train, dtype=torch.float32, device=DEVICE)
X_ptpn2_test_tensor  = torch.tensor(X_ptpn2_test,  dtype=torch.float32, device=DEVICE)
X_ctrl_tensor        = torch.tensor(X_ctrl_hvg,    dtype=torch.float32, device=DEVICE)

ae.train()
for epoch in range(AE_EPOCHS):
    perm = torch.randperm(n_ptpn2_train, device=DEVICE)
    total = 0.0
    for i in range(0, n_ptpn2_train, BATCH_SIZE):
        idx = perm[i:i+BATCH_SIZE]
        batch = X_ptpn2_train_tensor[idx]
        ae_opt.zero_grad()
        recon, _ = ae(batch)
        loss = ae_loss_fn(recon, batch)
        loss.backward()
        ae_opt.step()
        total += loss.item() * batch.size(0)
    if (epoch+1) % 5 == 0:
        print(f"[AE] Epoch {epoch+1}/{AE_EPOCHS} | Loss={total/n_ptpn2_train:.4f}")

ae.eval()
with torch.no_grad():
    Z_ptpn2_train = ae.encode(X_ptpn2_train_tensor).cpu().numpy()
    Z_ptpn2_test  = ae.encode(X_ptpn2_test_tensor).cpu().numpy()
    Z_ctrl        = ae.encode(X_ctrl_tensor).cpu().numpy()

print("Gene emb PTPN2 train:", Z_ptpn2_train.shape)
print("Gene emb PTPN2 test:",  Z_ptpn2_test.shape)
print("Gene emb CTRL:",        Z_ctrl.shape)


# ============================================================
# 6. PCA (PTPN2 TRAIN → OTHERS)
# ============================================================

pca_model = PCA(n_components=N_PCS)
X_pca_ptpn2_train = pca_model.fit_transform(Z_ptpn2_train)
X_pca_ptpn2_test  = pca_model.transform(Z_ptpn2_test)
X_pca_ctrl        = pca_model.transform(Z_ctrl)

print("PCA PTPN2 train:", X_pca_ptpn2_train.shape)
print("PCA PTPN2 test:",  X_pca_ptpn2_test.shape)
print("PCA CTRL:",        X_pca_ctrl.shape)


# ============================================================
# 7. SIMPLE INFERENCE-ONLY GAT ON JOINT PCA SPACE
# ============================================================

class SimpleGATLayer(nn.Module):
    def __init__(self, in_dim, out_dim, neighbor_lists, alpha=0.2):
        super().__init__()
        self.neighbors = neighbor_lists
        self.W = nn.Parameter(torch.Tensor(in_dim, out_dim))
        self.a = nn.Parameter(torch.Tensor(2*out_dim, 1))
        self.leaky = nn.LeakyReLU(alpha)
        nn.init.xavier_uniform_(self.W)
        nn.init.xavier_uniform_(self.a)

    def forward(self, X):
        Wh = X @ self.W
        out = torch.zeros_like(Wh)
        for i in range(Wh.shape[0]):
            neigh = self.neighbors[i]
            h_i = Wh[i]
            h_j = Wh[neigh]
            k = len(neigh)
            a_in = torch.cat([h_i.expand(k, -1), h_j], dim=1)
            e = self.leaky(a_in @ self.a).squeeze(-1)
            alpha = torch.softmax(e, dim=0)
            out[i] = (alpha.unsqueeze(1) * h_j).sum(0)
        return out

# Build combined PCA for graph
X_pca_full = np.vstack([X_pca_ptpn2_train, X_pca_ptpn2_test, X_pca_ctrl])
n_tr   = X_pca_ptpn2_train.shape[0]
n_te   = X_pca_ptpn2_test.shape[0]
n_ctrl = X_pca_ctrl.shape[0]

adata_pca_full = ad.AnnData(X_pca_full)
adata_pca_full.obsm["X_pca"] = X_pca_full
sc.pp.neighbors(adata_pca_full, n_neighbors=15, use_rep="X_pca")
A_full = adata_pca_full.obsp["connectivities"].tocsr()

neighbor_lists_full = []
for i in range(A_full.shape[0]):
    neigh = A_full[i].indices
    if i not in neigh:
        neigh = np.append(neigh, i)
    neighbor_lists_full.append(neigh)

gat_layer = SimpleGATLayer(N_PCS, GAT_OUT_DIM, neighbor_lists_full).to(DEVICE)

with torch.no_grad():
    X_gat_full = gat_layer(
        torch.tensor(X_pca_full, dtype=torch.float32, device=DEVICE)
    ).cpu().numpy()

X_gat_ptpn2_train = X_gat_full[:n_tr]
X_gat_ptpn2_test  = X_gat_full[n_tr:n_tr+n_te]
X_gat_ctrl        = X_gat_full[n_tr+n_te:]

print("GAT PTPN2 train:", X_gat_ptpn2_train.shape)
print("GAT PTPN2 test:",  X_gat_ptpn2_test.shape)
print("GAT CTRL:",        X_gat_ctrl.shape)


# ============================================================
# 8. FUSED EMBEDDINGS (SHARED LATENT SPACE)
# ============================================================

emb_ptpn2_train = np.concatenate([Z_ptpn2_train, X_pca_ptpn2_train, X_gat_ptpn2_train], axis=1)
emb_ptpn2_test  = np.concatenate([Z_ptpn2_test,  X_pca_ptpn2_test,  X_gat_ptpn2_test],  axis=1)
emb_ctrl        = np.concatenate([Z_ctrl,        X_pca_ctrl,        X_gat_ctrl],        axis=1)

emb_ptpn2_all = np.vstack([emb_ptpn2_train, emb_ptpn2_test])

print("Fused PTPN2 train:", emb_ptpn2_train.shape)
print("Fused PTPN2 test:",  emb_ptpn2_test.shape)
print("Fused CTRL:",        emb_ctrl.shape)


# ============================================================
# 9. PSEUDOTIME (PTPN2 TRAIN) + MAPPING TO PTPN2 TEST & CTRL
# ============================================================

# DPT on PTPN2 TRAIN (using HVG expression)
sc.pp.neighbors(adata_ptpn2_train, n_neighbors=15, use_rep=None)
sc.tl.diffmap(adata_ptpn2_train)
adata_ptpn2_train.uns["iroot"] = len(adata_ptpn2_train) // 2
sc.tl.dpt(adata_ptpn2_train)
pt_ptpn2_train = adata_ptpn2_train.obs["dpt_pseudotime"].values
print("PTPN2 train pseudotime:", pt_ptpn2_train.shape)

def map_to_pseudotime(ref_emb, ref_pt, X):
    d = cdist(X, ref_emb)
    nn = d.argmin(axis=1)
    return ref_pt[nn]

pt_ptpn2_test = map_to_pseudotime(emb_ptpn2_train, pt_ptpn2_train, emb_ptpn2_test)
pt_ctrl       = map_to_pseudotime(emb_ptpn2_train, pt_ptpn2_train, emb_ctrl)

print("PTPN2 test pseudotime (mapped):", pt_ptpn2_test.shape)
print("CTRL pseudotime (mapped):",       pt_ctrl.shape)

# -------- Pseudotime cleaning (avoid NaNs/Inf) --------

def clean_pseudotime(pt):
    pt = np.array(pt, dtype=float)
    mask = np.isfinite(pt)
    if mask.sum() == 0:
        raise ValueError("All pseudotime values invalid after mapping!")
    mean_val = np.nanmean(pt[mask])
    pt[~mask] = mean_val
    return pt

pt_ptpn2_train = clean_pseudotime(pt_ptpn2_train)
pt_ptpn2_test  = clean_pseudotime(pt_ptpn2_test)
pt_ctrl        = clean_pseudotime(pt_ctrl)

pt_ptpn2_all = np.concatenate([pt_ptpn2_train, pt_ptpn2_test], axis=0)


# ============================================================
# 10. WORLD MODEL f_θ (PTPN2 TRAIN, kNN NEXT-STEP)
# ============================================================

def build_world_model_pairs(emb, pt, k=5, min_delta=1e-4):
    n = emb.shape[0]
    X_curr, X_next = [], []
    for i in range(n):
        mask = pt > pt[i] + min_delta
        cand_idx = np.where(mask)[0]
        if len(cand_idx) < k:
            continue
        cand_emb = emb[cand_idx]
        dists = np.linalg.norm(cand_emb - emb[i], axis=1)
        nn_rel = np.argsort(dists)[:k]
        nn_abs = cand_idx[nn_rel]
        x_next = emb[nn_abs].mean(axis=0)
        X_curr.append(emb[i])
        X_next.append(x_next)
    X_curr = np.vstack(X_curr)
    X_next = np.vstack(X_next)
    return X_curr, X_next

X_curr_wm, X_next_wm = build_world_model_pairs(
    emb_ptpn2_train, pt_ptpn2_train, k=K_NEXT
)
print("World model pairs:", X_curr_wm.shape, X_next_wm.shape)

class WorldModel(nn.Module):
    def __init__(self, dim, hidden=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, dim)
        )

    def forward(self, x):
        return self.net(x)

world_dim = emb_ptpn2_train.shape[1]
world_model = WorldModel(world_dim, WORLD_HIDDEN).to(DEVICE)
world_opt = torch.optim.Adam(world_model.parameters(), lr=WORLD_LR)
world_loss_fn = nn.MSELoss()

X_curr_tensor = torch.tensor(X_curr_wm, dtype=torch.float32, device=DEVICE)
X_next_tensor = torch.tensor(X_next_wm, dtype=torch.float32, device=DEVICE)

world_model.train()
for epoch in range(WORLD_EPOCHS):
    perm = torch.randperm(X_curr_wm.shape[0], device=DEVICE)
    total = 0.0
    for i in range(0, X_curr_wm.shape[0], BATCH_SIZE):
        idx = perm[i:i+BATCH_SIZE]
        batch_curr = X_curr_tensor[idx]
        batch_next = X_next_tensor[idx]
        world_opt.zero_grad()
        pred_next = world_model(batch_curr)
        loss = world_loss_fn(pred_next, batch_next)
        loss.backward()
        world_opt.step()
        total += loss.item() * batch_curr.size(0)
    print(f"[WORLD] Epoch {epoch+1}/{WORLD_EPOCHS} | Loss={total/X_curr_wm.shape[0]:.4f}")

world_model.eval()




# ============================================================
# AUTOMATIC ACTION-SCALE MATCHING (CTRL ↔ PTPN2)
# ============================================================

def estimate_worldmodel_step(world_model, emb, n_samples=500):
    """
    Estimate average |fθ(x) - x| step magnitude in PTPN2.
    """
    idx = np.random.choice(emb.shape[0], size=n_samples, replace=False)
    xs = emb[idx]

    diffs = []
    with torch.no_grad():
        for x in xs:
            s = torch.tensor(x, dtype=torch.float32, device=DEVICE).unsqueeze(0)
            next_s = world_model(s).cpu().numpy()[0]
            diffs.append(np.linalg.norm(next_s - x))

    return float(np.mean(diffs))


def compute_optimal_action_scale(world_model, emb_ctrl, emb_ptpn2_train):
    """
    Compute CTRL action scale so that:
        mean step in CTRL ≈ mean step in PTPN2 world model
    """
    D = emb_ctrl.shape[1]

    # --- 1. estimate world model step ---
    delta_ptpn2 = estimate_worldmodel_step(world_model, emb_ptpn2_train)

    # --- 2. expected norm of random [-1,1]^D action ---
    # Var(action_i) = 1/3 → expected squared norm = D/3
    expected_norm_random = math.sqrt(D / 3)

    # --- 3. solve scale such that:
    #       scale * expected_norm_random = delta_ptpn2
    scale_opt = delta_ptpn2 / expected_norm_random

    return float(scale_opt)


ACTION_SCALE  = compute_optimal_action_scale(
    world_model,
    emb_ctrl,
    emb_ptpn2_train
)

#print("\n=== AUTO-MATCHED ACTION SCALE ===")
#print("Computed optimal scale:", AUTO_SCALE)
#print("Original (manual) scale:", ACTION_SCALE)














# ============================================================
# 11. TARGETS + CTRL 80/20 SPLIT
# ============================================================

# PTPN2 real target
target_ptpn2_real = emb_ptpn2_train.mean(axis=0)
print("Target latent dim (PTPN2):", target_ptpn2_real.shape)

# CTRL 80/20 split in fused latent space for model-free RL
emb_ctrl_train, emb_ctrl_test, pt_ctrl_train, pt_ctrl_test = train_test_split(
    emb_ctrl, pt_ctrl, test_size=0.20, random_state=42, shuffle=True
)

mu_ctrl_train  = emb_ctrl_train.mean(axis=0)
std_ctrl_train = emb_ctrl_train.std(axis=0)
insilico_target_ctrl = mu_ctrl_train - ALPHA * std_ctrl_train


# ============================================================
# 12. MODEL-BASED RL ENV (PTPN2WorldEnv)
# ============================================================

class PTPN2WorldEnv(gym.Env):
    """
    Model-based RL on PTPN2:
      - State: fused PTPN2 latent
      - t=0: RL applies one perturbation (action)
      - t>0: state evolves via world model f_θ
      - Reward: -||state - target||^2
    """
    metadata = {"render_modes": ["human"]}

    def __init__(self, emb, target, world_model, horizon=12, action_scale=0.3):
        super().__init__()
        self.emb = emb.astype(np.float32)
        self.target = target.astype(np.float32)
        self.world_model = world_model
        self.horizon = horizon
        self.action_scale = action_scale
        self.n, self.dim = self.emb.shape

        self.action_space = spaces.Box(-1.0, 1.0, (self.dim,), np.float32)
        self.observation_space = spaces.Box(-np.inf, np.inf, (self.dim,), np.float32)

        self.state = None
        self.t = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        idx = np.random.randint(0, self.n)
        self.state = self.emb[idx].copy()
        self.t = 0
        return self.state, {}

    def step(self, action):
        if self.t == 0:
            # CRISPR OFF → no perturbation applied
            next_state = self.state.copy()
        else:
            # Always world-model evolution
            with torch.no_grad():
                s = torch.tensor(self.state, dtype=torch.float32, device=DEVICE).unsqueeze(0)
                next_state = self.world_model(s).cpu().numpy()[0]

        diff = next_state - self.target
        reward = float(-np.dot(diff, diff))

        self.state = next_state
        self.t += 1
        terminated = self.t >= self.horizon
        truncated = False
        return next_state, reward, terminated, truncated, {}




env_ptpn2_train = PTPN2WorldEnv(
    emb_ptpn2_train, target_ptpn2_real, world_model,
    horizon=HORIZON, action_scale=ACTION_SCALE
)
env_ptpn2_test  = PTPN2WorldEnv(
    emb_ptpn2_test,  target_ptpn2_real, world_model,
    horizon=HORIZON, action_scale=ACTION_SCALE
)

check_env(env_ptpn2_train)


# ============================================================
# 13. MODEL-FREE RL ENV (CRISPRCtrlEnv) ON CTRL
# ============================================================

class CRISPRCtrlEnvCytokine(gym.Env):
    """
    CTRL model-free RL with cytokine + immunotherapy microenvironment.

    Biology encoded:
      - Anti–PD-1 -> boosts effector function, reduces exhaustion penalty
      - Anti–CTLA-4 -> enhances priming, expands action effect scale
      - Cytokine field -> shapes both reward and state transition
    """

    metadata = {"render_modes": ["human"]}

    def __init__(
        self,
        ctrl_emb,
        ptime,
        target,
        cytokines,
        anti_pd1=False,
        anti_ctla4=False,
        scale=0.3,
        horizon=12
    ):
        super().__init__()

        # embeddings & pseudotime
        self.ctrl = ctrl_emb.astype(np.float32)
        self.pseudotime = np.array(ptime, dtype=float)
        self.target = target.astype(np.float32)

        # cytokines: dict e.g.
        # {'IL2':1.2, 'IL7':0.8, 'IFNG':1.5, 'IL10':0.4}
        self.cyt = cytokines

        self.anti_pd1 = anti_pd1
        self.anti_ctla4 = anti_ctla4

        # base scale & horizon
        self.base_scale = scale
        self.horizon = horizon
        self.n_cells, self.dim = self.ctrl.shape

        self.action_space = spaces.Box(-1.0, 1.0, (self.dim,), np.float32)
        self.observation_space = spaces.Box(-np.inf, np.inf, (self.dim,), np.float32)

        self.state = None
        self.t = 0

    # ----------------------------------------------------------
    # IMMUNOTHERAPY & CYTOKINE MODULATION
    # ----------------------------------------------------------

    def compute_transition_scale(self):
        """Cytokine + CTLA-4 effects on how strongly actions move the state."""

        scale = self.base_scale

        # IL-2, IL-7, IL-15 -> expansion & persistence
        scale *= 1.0 + 0.25 * (self.cyt.get('IL2',0)
                               + self.cyt.get('IL7',0)
                               + self.cyt.get('IL15',0))

        # IFN-gamma -> increases activation bias
        scale *= 1.0 + 0.15 * self.cyt.get('IFNG', 0)

        # IL-10 -> suppressive
        scale *= 1.0 - 0.20 * self.cyt.get('IL10', 0)

        # CTLA-4 BLOCKADE -> stronger priming, stronger response
        if self.anti_ctla4:
            scale *= 1.35

        return float(max(scale, 0.01))

    def compute_reward_modulation(self, diff, action):
        """Reward shaped by cytokines & PD-1 pathway."""

        # BASE reward
        reward = -np.dot(diff, diff)

        # --- CYTOKINES --- #

        # IFN-gamma promotes target-directed movement
        reward *= 1.0 + 0.20 * self.cyt.get('IFNG', 0)

        # IL-2 improves expansion: reward gets less negative
        reward *= 1.0 + 0.10 * self.cyt.get('IL2', 0)

        # IL-10 suppresses movement toward the target (immunoregulatory)
        reward *= 1.0 - 0.20 * self.cyt.get('IL10', 0)

        # --- ANTIBODY THERAPIES --- #

        if self.anti_pd1:
            # PD-1 blockade removes exhaustion → better reward
            reward *= 1.25

        if self.anti_ctla4:
            # CTLA-4 blockade → amplifies priming → more reward
            reward *= 1.15

        # SMALL penalty for action magnitude
        reward -= 0.01 * np.linalg.norm(action)

        return float(reward)

    # ----------------------------------------------------------
    # RESET: pseudotime-weighted sampling
    # ----------------------------------------------------------

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        probs = np.array(self.pseudotime, dtype=float)
        probs[~np.isfinite(probs)] = 0.0
        probs = probs + 1e-6
        total = probs.sum()

        if total == 0 or not np.isfinite(total):
            probs = np.ones_like(probs) / len(probs)
        else:
            probs = probs / total

        idx = np.random.choice(self.n_cells, p=probs)
        self.state = self.ctrl[idx].copy()
        self.t = 0
        return self.state, {}

    # ----------------------------------------------------------
    # STEP FUNCTION
    # ----------------------------------------------------------

    def step(self, action):
        action = np.clip(action, -1, 1)

        transition_scale = self.compute_transition_scale()
        next_state = self.state + transition_scale * action

        diff = next_state - self.target

        reward = self.compute_reward_modulation(diff, action)

        self.state = next_state
        self.t += 1

        terminated = self.t >= self.horizon
        truncated = False
        return next_state, reward, terminated, truncated, {}




#cytokines = {
 #   "IL2": 1.4,
#    "IL7": 1.2,
#    "IL15": 1.0,
#    "IFNG": 1.8,
#    "TNF": 0.7,
#    "IL10": 0.0
#}
import pandas as pd
import scipy.io
import numpy as np

def extract_cytokines_auto(genes_path, matrix_path):
    """
    Automatically extracts cytokine expression (mean per cell)
    and normalizes them for RL environment use.
    """

    # Cytokines of interest
    cytokine_list = ["Il2", "Il7", "Il15", "Ifng", "Tnf", "Il10"]

    # -----------------------------
    # Load files
    # -----------------------------
    genes = pd.read_csv(genes_path, sep="\t", header=None)
    gene_names = genes[1].tolist()

    mtx = scipy.io.mmread(matrix_path).tocsc()

    # -----------------------------
    # Extract expression
    # -----------------------------
    raw_expr = {}
    for cyt in cytokine_list:
        if cyt in gene_names:
            idx = gene_names.index(cyt)
            arr = mtx[idx, :].toarray().squeeze()
            raw_expr[cyt] = float(arr.mean())
        else:
            raw_expr[cyt] = 0.0

    # -----------------------------
    # Normalize
    # -----------------------------
    max_val = max(raw_expr.values()) if max(raw_expr.values()) > 0 else 1.0
    norm_expr = {cyt: raw_expr[cyt] / max_val for cyt in cytokine_list}

    # -----------------------------
    # RL-ready cytokine dict (uppercase)
    # -----------------------------
    cytokines = {
        "IL2":  norm_expr["Il2"],
        "IL7":  norm_expr["Il7"],
        "IL15": norm_expr["Il15"],
        "IFNG": norm_expr["Ifng"],
        "TNF":  norm_expr["Tnf"],
        "IL10": norm_expr["Il10"],
    }

    return cytokines, raw_expr, norm_expr


cytokines, raw_expr, norm_expr = extract_cytokines_auto(PTPN2_FEAT, PTPN2_MATRIX)




env_ctrl_train = CRISPRCtrlEnvCytokine(
    emb_ctrl_train, pt_ctrl_train,
    insilico_target_ctrl,
    cytokines,
    anti_pd1=True,
    anti_ctla4=True,
    scale=ACTION_SCALE,
    horizon=HORIZON
)

env_ctrl_test  = CRISPRCtrlEnvCytokine(
    emb_ctrl_test,  pt_ctrl_test,
    target_ptpn2_real, cytokines,
    anti_pd1=True,
    anti_ctla4=True,
    scale=ACTION_SCALE,
    horizon=HORIZON
)

check_env(env_ctrl_train)




# ============================================================
# KL Extraction helper (shared)
# ============================================================

def estimate_trpo_kl(trpo_model):
    """
    Extract KL divergence approximation from TRPO logger.
    If key not found, return safe default KL.
    """
    try:
        return float(trpo_model.logger.name_to_value["train/approx_kl"])
    except:
        return 0.01  # safe fallback



# ============================================================
# 14. RL TRAINING: PPO / TRPO / TRPO→PPO
# ============================================================

# ---- CTRL model-free RL ----
ppo_ctrl = PPO("MlpPolicy", env_ctrl_train, verbose=1)
ppo_ctrl.learn(PPO_TIMESTEPS_BASELINE_ctrl)

trpo_ctrl = TRPO("MlpPolicy", env_ctrl_train, verbose=1)
trpo_ctrl.learn(TRPO_TIMESTEPS_STAGE1_ctrl)

target_kl_trpo_ctrl = estimate_trpo_kl(trpo_ctrl)

ppo_ctrl_ms = PPO(
    "MlpPolicy",
    env_ctrl_train,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=256,
    gae_lambda=0.95,
    gamma=0.99,
    clip_range=0.2,                        # still used
    target_kl=target_kl_trpo_ctrl * 1.5,        # ← KL-aware warm-start!
    clip_range_vf=None,
)

#ppo_ctrl_ms = PPO("MlpPolicy", env_ctrl_train, verbose=1)
ppo_ctrl_ms.policy.load_state_dict(trpo_ctrl.policy.state_dict())
ppo_ctrl_ms.learn(PPO_TIMESTEPS_STAGE2_ctrl)

# ---- PTPN2 model-based RL ----
ppo_ptpn2 = PPO("MlpPolicy", env_ptpn2_train, verbose=1)
ppo_ptpn2.learn(PPO_TIMESTEPS_BASELINE)

trpo_ptpn2 = TRPO("MlpPolicy", env_ptpn2_train, verbose=1)
trpo_ptpn2.learn(TRPO_TIMESTEPS_STAGE1)
target_kl_trpo_ptpn2 = estimate_trpo_kl(trpo_ptpn2)
#ppo_ptpn2_ms = PPO("MlpPolicy", env_ptpn2_train, verbose=1)

ppo_ptpn2_ms = PPO(
    "MlpPolicy",
    env_ptpn2_train,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=256,
    target_kl=target_kl_trpo_ptpn2 * 1.5,
)

ppo_ptpn2_ms.policy.load_state_dict(trpo_ptpn2.policy.state_dict())
ppo_ptpn2_ms.learn(PPO_TIMESTEPS_STAGE2)


# ============================================================
# 15. MULTI-STEP ROLLOUTS (START vs FINAL) FOR METRICS
# ============================================================

def rollout_last(agent, env, n_samples=1000):
    raw, shifted, rewards = [], [], []
    for _ in range(n_samples):
        obs, _ = env.reset()
        start = obs.copy()
        total_r = 0.0
        for _ in range(env.horizon):
            a, _ = agent.predict(obs, deterministic=True)
            obs, r, done, _, _ = env.step(a)
            total_r += r
            if done:
                break
        raw.append(start)
        shifted.append(obs.copy())
        rewards.append(total_r)
    return np.vstack(raw), np.vstack(shifted), np.array(rewards)

# CTRL env
raw_ctrl_ppo_tr,  shift_ctrl_ppo_tr,  rew_ctrl_ppo_tr  = rollout_last(ppo_ctrl,    env_ctrl_train, ROLLOUT_SAMPLES)
raw_ctrl_ppo_te,  shift_ctrl_ppo_te,  rew_ctrl_ppo_te  = rollout_last(ppo_ctrl,    env_ctrl_test,  ROLLOUT_SAMPLES)
raw_ctrl_ms_tr,   shift_ctrl_ms_tr,   rew_ctrl_ms_tr   = rollout_last(ppo_ctrl_ms, env_ctrl_train, ROLLOUT_SAMPLES)
raw_ctrl_ms_te,   shift_ctrl_ms_te,   rew_ctrl_ms_te   = rollout_last(ppo_ctrl_ms, env_ctrl_test,  ROLLOUT_SAMPLES)
#raw_ctrl_trpo_tr, shift_ctrl_trpo_tr, rew_ctrl_trpo_tr = rollout_last(trpo_ctrl,   env_ctrl_train, ROLLOUT_SAMPLES)
#raw_ctrl_trpo_te, shift_ctrl_trpo_te, rew_ctrl_trpo_te = rollout_last(trpo_ctrl,   env_ctrl_test,  ROLLOUT_SAMPLES)

# PTPN2 env
raw_ptpn2_ppo_tr,  shift_ptpn2_ppo_tr,  rew_ptpn2_ppo_tr  = rollout_last(ppo_ptpn2,    env_ptpn2_train, ROLLOUT_SAMPLES)
raw_ptpn2_ppo_te,  shift_ptpn2_ppo_te,  rew_ptpn2_ppo_te  = rollout_last(ppo_ptpn2,    env_ptpn2_test,  ROLLOUT_SAMPLES)
raw_ptpn2_ms_tr,   shift_ptpn2_ms_tr,   rew_ptpn2_ms_tr   = rollout_last(ppo_ptpn2_ms, env_ptpn2_train, ROLLOUT_SAMPLES)
raw_ptpn2_ms_te,   shift_ptpn2_ms_te,   rew_ptpn2_ms_te   = rollout_last(ppo_ptpn2_ms, env_ptpn2_test,  ROLLOUT_SAMPLES)
#raw_ptpn2_trpo_tr, shift_ptpn2_trpo_tr, rew_ptpn2_trpo_tr = rollout_last(trpo_ptpn2,   env_ptpn2_train, ROLLOUT_SAMPLES)
#raw_ptpn2_trpo_te, shift_ptpn2_trpo_te, rew_ptpn2_trpo_te = rollout_last(trpo_ptpn2,   env_ptpn2_test,  ROLLOUT_SAMPLES)


# ============================================================
# 16. METRICS (PER DIM + MEAN)
# ============================================================

def metrics_1d(true_vals, pred_vals):
    m = {}
    y_true = (true_vals > 0).astype(int)
    y_pred = (pred_vals > 0).astype(int)
   # m["Accuracy"]  = accuracy_score(y_true, y_pred)
   # m["Precision"] = precision_score(y_true, y_pred, zero_division=0)
   # m["Recall"]    = recall_score(y_true, y_pred, zero_division=0)
   # m["F1"]        = f1_score(y_true, y_pred, zero_division=0)
   # try:
   #     m["AUPRC"] = average_precision_score(y_true, y_pred)
   # except Exception:
  #      m["AUPRC"] = np.nan
    mse = mean_squared_error(true_vals, pred_vals)
    m["MSE"]  = mse
    m["RMSE"] = math.sqrt(mse)
    m["MAE"]  = mean_absolute_error(true_vals, pred_vals)
    m["R2"]   = r2_score(true_vals, pred_vals)
    if np.std(true_vals) == 0:
        m["Pearson"] = 0.0
    else:
        m["Pearson"] = np.corrcoef(true_vals, pred_vals)[0, 1]
    return m

def eval_model(raw, shifted, algo_name, system_name):
    rows = []
    D = raw.shape[1]
    for d in range(D):
        md = metrics_1d(raw[:, d], shifted[:, d])
        md["Dim"] = d
        md["Algorithm"] = algo_name
        md["System"] = system_name
        rows.append(md)
    return pd.DataFrame(rows)

dfs = []

# CTRL
dfs.append(eval_model(raw_ctrl_ppo_tr,  shift_ctrl_ppo_tr,  "PPO (Train)",      "CTRL"))
dfs.append(eval_model(raw_ctrl_ppo_te,  shift_ctrl_ppo_te,  "PPO (Test)",       "CTRL"))
dfs.append(eval_model(raw_ctrl_ms_tr,   shift_ctrl_ms_tr,   "TRPO→PPO (Train)", "CTRL"))
dfs.append(eval_model(raw_ctrl_ms_te,   shift_ctrl_ms_te,   "TRPO→PPO (Test)",  "CTRL"))
#dfs.append(eval_model(raw_ctrl_trpo_tr, shift_ctrl_trpo_tr, "TRPO (Train)",     "CTRL"))
#dfs.append(eval_model(raw_ctrl_trpo_te, shift_ctrl_trpo_te, "TRPO (Test)",      "CTRL"))

# PTPN2
dfs.append(eval_model(raw_ptpn2_ppo_tr,  shift_ptpn2_ppo_tr,  "PPO (Train)",      "PTPN2"))
dfs.append(eval_model(raw_ptpn2_ppo_te,  shift_ptpn2_ppo_te,  "PPO (Test)",       "PTPN2"))
dfs.append(eval_model(raw_ptpn2_ms_tr,   shift_ptpn2_ms_tr,   "TRPO→PPO (Train)", "PTPN2"))
dfs.append(eval_model(raw_ptpn2_ms_te,   shift_ptpn2_ms_te,   "TRPO→PPO (Test)",  "PTPN2"))
#dfs.append(eval_model(raw_ptpn2_trpo_tr, shift_ptpn2_trpo_tr, "TRPO (Train)",     "PTPN2"))
#dfs.append(eval_model(raw_ptpn2_trpo_te, shift_ptpn2_trpo_te, "TRPO (Test)",      "PTPN2"))

df_all = pd.concat(dfs, axis=0)
df_all.to_csv("unified_ctrl_ptpn2_rl_single_dim_metrics.csv", index=False)

#metric_cols = ["Accuracy","Precision","Recall","F1","AUPRC","MSE","RMSE","MAE","R2","Pearson"]
metric_cols = ["MSE","RMSE","MAE","R2","Pearson"]
df_mean = df_all.groupby(["System","Algorithm"])[metric_cols].mean()
df_mean.to_csv("unified_ctrl_ptpn2_rl_mean_metrics.csv")

print("\n=== MEAN METRICS (by System, Algorithm) ===")
print(df_mean)


# ============================================================
# 17. DISTANCE TO REAL PTPN2 MEAN (TEST TRAJECTORIES)
# ============================================================

def mean_dist(X, target):
    return np.linalg.norm(X - target[None, :], axis=1).mean()

print("\n=== Distance to REAL PTPN2 TRAIN mean (TRPO→PPO, Test) ===")
print("CTRL TRPO→PPO:",  mean_dist(shift_ctrl_ms_te,   target_ptpn2_real))
print("PTPN2 TRPO→PPO:", mean_dist(shift_ptpn2_ms_te, target_ptpn2_real))


# ============================================================
# 18. PSEUDOTIME ALIGNMENT FOR SIMULATED TRAJECTORIES
# ============================================================

pt_ctrl_ms_test   = map_to_pseudotime(emb_ptpn2_train, pt_ptpn2_train, shift_ctrl_ms_te)
pt_ptpn2_ms_test  = map_to_pseudotime(emb_ptpn2_train, pt_ptpn2_train, shift_ptpn2_ms_te)

print("\n=== Pseudotime alignment (TRPO→PPO, Test) ===")
print(f"CTRL TRPO→PPO:   mean={pt_ctrl_ms_test.mean():.3f}, std={pt_ctrl_ms_test.std():.3f}")
print(f"PTPN2 TRPO→PPO:  mean={pt_ptpn2_ms_test.mean():.3f}, std={pt_ptpn2_ms_test.std():.3f}")


# ============================================================
# 19. FULL H-STEP TRAJECTORIES FOR IQCELL-STYLE COMPARISON
# ============================================================

def rollout_trajectories(agent, env, n_traj):
    """
    Collect full trajectories: shape (n_traj, horizon+1, dim)
    """
    trajs = []
    for _ in range(n_traj):
        obs, _ = env.reset()
        traj = [obs.copy()]
        for _ in range(env.horizon):
            a, _ = agent.predict(obs, deterministic=True)
            obs, r, done, _, _ = env.step(a)
            traj.append(obs.copy())
            if done:
                break
        # pad if shorter than horizon+1
        if len(traj) < env.horizon + 1:
            last = traj[-1].copy()
            while len(traj) < env.horizon + 1:
                traj.append(last.copy())
        trajs.append(np.stack(traj, axis=0))
    return np.stack(trajs, axis=0)  # (N, T, D)

# TRPO→PPO trajectories on TEST envs
traj_ctrl_ms_te  = rollout_trajectories(ppo_ctrl_ms,  env_ctrl_test,  N_TRAJ_DTW)
traj_ptpn2_ms_te = rollout_trajectories(ppo_ptpn2_ms, env_ptpn2_test, N_TRAJ_DTW)

traj_ctrl_mean   = traj_ctrl_ms_te.mean(axis=0)    # (T, D)
traj_ptpn2_mean  = traj_ptpn2_ms_te.mean(axis=0)   # (T, D)


# ============================================================
# PPO TRAJECTORIES (TEST ENVS) —
# ============================================================

traj_ctrl_ppo_te  = rollout_trajectories(ppo_ctrl,    env_ctrl_test,  N_TRAJ_DTW)
traj_ptpn2_ppo_te = rollout_trajectories(ppo_ptpn2,   env_ptpn2_test, N_TRAJ_DTW)

traj_ctrl_mean_ppo   = traj_ctrl_ppo_te.mean(axis=0)
traj_ptpn2_mean_ppo  = traj_ptpn2_ppo_te.mean(axis=0)


# ============================================================
# 20. EXPERIMENTAL TRAJECTORIES (PSEUDOTIME-SORTED)
# ============================================================

def build_exp_trajectory(emb, pt, n_points):
    """
    Build experimental trajectory by:
      - sorting cells by pseudotime
      - sampling n_points along that ordering
    """
    order = np.argsort(pt)
    emb_sorted = emb[order]
    L = emb_sorted.shape[0]
    idx = np.linspace(0, L-1, n_points).astype(int)
    return emb_sorted[idx]

T_traj = HORIZON + 1

exp_traj_ctrl  = build_exp_trajectory(emb_ctrl,       pt_ctrl,        T_traj)
exp_traj_ptpn2 = build_exp_trajectory(emb_ptpn2_all,  pt_ptpn2_all,   T_traj)


# ============================================================
# 21. DTW + WASSERSTEIN (OT) TRAJECTORY SIMILARITY
# ============================================================


def dtw_distance_1d(x, y):
    """Simple 1D DTW distance."""
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    n, m = len(x), len(y)
    D = np.full((n+1, m+1), np.inf)
    D[0, 0] = 0.0
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(x[i-1] - y[j-1])
            D[i, j] = cost + min(D[i-1, j], D[i, j-1], D[i-1, j-1])
    return D[n, m]


# ------------------------------------------------------------
# Dictionary of algorithm → simulated trajectories
# ------------------------------------------------------------
algo_trajs = {
    ("CTRL",  "PPO"):       traj_ctrl_ppo_te,
    ("CTRL",  "TRPO→PPO"):  traj_ctrl_ms_te,
    ("PTPN2", "PPO"):       traj_ptpn2_ppo_te,
    ("PTPN2", "TRPO→PPO"):  traj_ptpn2_ms_te,
}

# Experimental real trajectories
exp_trajs = {
    "CTRL":  exp_traj_ctrl,
    "PTPN2": exp_traj_ptpn2,
}


results_traj = []

# ------------------------------------------------------------
# Compute DTW + Wasserstein for every algorithm × system
# ------------------------------------------------------------
for (system, algo), traj_sim in algo_trajs.items():

    traj_sim_mean = traj_sim.mean(axis=0)   # (T, D)
    traj_exp      = exp_trajs[system]       # (T, D)
    D_latent      = traj_sim_mean.shape[1]

    for d in range(D_latent):
        sim_seq = traj_sim_mean[:, d]
        exp_seq = traj_exp[:, d]
        dtw_val = dtw_distance_1d(sim_seq, exp_seq)
        wdist   = wasserstein_distance(sim_seq, exp_seq)

        results_traj.append({
            "System": system,
            "Algorithm": algo,
            "Dim": d,
            "DTW": dtw_val,
            "Wasserstein": wdist
        })


df_traj = pd.DataFrame(results_traj)
df_traj.to_csv("trajectory_similarity_dtw_wasserstein_all_algorithms.csv", index=False)

print("\n=== Trajectory similarity (mean DTW / Wasserstein per algorithm) ===")
df_avg = df_traj.groupby(["System","Algorithm"])[["DTW","Wasserstein"]].mean()
print(df_avg)

df_avg.to_csv("trajectory_similarity_average_by_algorithm.csv")



# ============================================================
# 22. DTW HEATMAP (DIMENSION-WISE, CTRL vs PTPN2)
# ============================================================

dtw_ctrl  = df_traj[df_traj["System"]=="CTRL"].sort_values("Dim")["DTW"].values
dtw_ptpn2 = df_traj[df_traj["System"]=="PTPN2"].sort_values("Dim")["DTW"].values

# ensure same length
D_min = min(len(dtw_ctrl), len(dtw_ptpn2))
dtw_mat = np.vstack([dtw_ctrl[:D_min], dtw_ptpn2[:D_min]])

plt.figure(figsize=(10, 4))
plt.imshow(dtw_mat, aspect="auto")
plt.yticks([0,1], ["CTRL", "PTPN2"])
plt.xlabel("Latent dimension")
plt.colorbar(label="DTW distance")
plt.title("DTW trajectory distance per latent dimension (CTRL vs PTPN2)")
plt.tight_layout()
plt.show()


# ============================================================
# 23. 2D VISUALIZATION OF REAL vs SIMULATED MANIFOLDS
# ============================================================

X_vis = np.vstack([
    emb_ctrl,
    emb_ptpn2_all,
    shift_ctrl_ms_te,
    shift_ptpn2_ms_te
])
pca_vis = PCA(n_components=2)
X_vis_2d = pca_vis.fit_transform(X_vis)

n_ctrl_real   = emb_ctrl.shape[0]
n_ptpn2_real  = emb_ptpn2_all.shape[0]
n_sim_ctrl    = shift_ctrl_ms_te.shape[0]
n_sim_ptpn2   = shift_ptpn2_ms_te.shape[0]

idx_ctrl_real   = np.arange(0, n_ctrl_real)
idx_ptpn2_real  = np.arange(n_ctrl_real, n_ctrl_real + n_ptpn2_real)
idx_sim_ctrl    = np.arange(n_ctrl_real + n_ptpn2_real,
                            n_ctrl_real + n_ptpn2_real + n_sim_ctrl)
idx_sim_ptpn2   = np.arange(n_ctrl_real + n_ptpn2_real + n_sim_ctrl,
                            n_ctrl_real + n_ptpn2_real + n_sim_ctrl + n_sim_ptpn2)

plt.figure(figsize=(9, 8))
plt.scatter(X_vis_2d[idx_ctrl_real,0],  X_vis_2d[idx_ctrl_real,1],
            s=5, alpha=0.2, label="CTRL (real)")
plt.scatter(X_vis_2d[idx_ptpn2_real,0], X_vis_2d[idx_ptpn2_real,1],
            s=5, alpha=0.4, label="PTPN2 (real)")
plt.scatter(X_vis_2d[idx_sim_ctrl,0],   X_vis_2d[idx_sim_ctrl,1],
            s=10, alpha=0.9, label="CTRL TRPO→PPO (sim)")
plt.scatter(X_vis_2d[idx_sim_ptpn2,0],  X_vis_2d[idx_sim_ptpn2,1],
            s=10, alpha=0.9, label="PTPN2 TRPO→PPO (sim)")

plt.xlabel("Latent PC1")
plt.ylabel("Latent PC2")
plt.title("Unified latent space: CTRL/PTPN2 real vs RL-simulated trajectories")
plt.legend()
plt.tight_layout()
plt.show()

Using device: cpu
CTRL raw: (3840, 27998) PTPN2 raw: (3966, 27998)
Common genes: 27998


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


PTPN2 train raw: (3172, 27998)
PTPN2 test raw: (794, 27998)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


HVG genes (PTPN2 train): 50
PTPN2 train HVG: (3172, 50)
PTPN2 test HVG: (794, 50)
CTRL HVG: (3840, 50)
Autoencoder input: (3172, 50)


...

PTPN2 train pseudotime: (3172,)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


PTPN2 test pseudotime (mapped): (794,)
CTRL pseudotime (mapped): (3840,)
World model pairs: (3166, 58) (3166, 58)
[WORLD] Epoch 1/100 | Loss=2.6903
[WORLD] Epoch 2/100 | Loss=1.8006
[WORLD] Epoch 3/100 | Loss=1.1049
[WORLD] Epoch 4/100 | Loss=0.7361
[WORLD] Epoch 5/100 | Loss=0.5841
[WORLD] Epoch 6/100 | Loss=0.5190
[WORLD] Epoch 7/100 | Loss=0.4842
[WORLD] Epoch 8/100 | Loss=0.4602
[WORLD] Epoch 9/100 | Loss=0.4404
[WORLD] Epoch 10/100 | Loss=0.4202
[WORLD] Epoch 11/100 | Loss=0.4045
[WORLD] Epoch 12/100 | Loss=0.3917
[WORLD] Epoch 13/100 | Loss=0.3840
[WORLD] Epoch 14/100 | Loss=0.3763
[WORLD] Epoch 15/100 | Loss=0.3656
[WORLD] Epoch 16/100 | Loss=0.3584
[WORLD] Epoch 17/100 | Loss=0.3547
[WORLD] Epoch 18/100 | Loss=0.3475
[WORLD] Epoch 19/100 | Loss=0.3433
[WORLD] Epoch 20/100 | Loss=0.3397
[WORLD] Epoch 21/100 | Loss=0.3368
[WORLD] Epoch 22/100 | Loss=0.3312
[WORLD] Epoch 23/100 | Loss=0.3271
[WORLD] Epoch 24/100 | Loss=0.3258
[WORLD] Epoch 25/100 | Loss=0.3229
[WORLD] Epoch 26/100

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -884     |
| time/              |          |
|    fps             | 1015     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -828       |
| time/                   |            |
|    fps                  | 711        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01894423 |
|    clip_fraction        | 0.264      |
|    clip_range           | 0.2        |
|    entropy_loss         | -82.3

In [5]:
pip install sb3_contrib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 3.1 MB/s eta 0:00:00
